### Step 1: extracting dialogs and explicit speakers from raw text (deterministic method)

In [87]:
raw_text_file = open('corpus/PRIDPREJ_NONEWLINE_Organize_v2.txt', 'r+')

In [88]:
import re
import nltk
from nltk.parse import stanford

# Add the jar and model via their path (instead of setting environment variables):
#jar = '.\stanford-parser-full-2015-01-30\stanford-parser.jar'
#model = '.\stanford-parser-full-2015-01-30\stanford-parser.jar'

jar = './stanford-parser-full-2017-06-09/stanford-parser-3.8.0.jar'
model = './stanford-parser-full-2017-06-09/stanford-parser-3.8.0.jar'

dep_parser = stanford.StanfordDependencyParser(model, jar, model_path='edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz', encoding='utf8')

line = raw_text_file.readline()

regex_utterance = r'\`\`(.+?)\'\''
regex_narration_inbetween = r'\'\'(.+?)\`\`'
regex_narration_end = r'\'\'(.+?)$'
regex_narration = r'\'\'(.+?)(?:(?:\`\`)|$)'
verbs = ['said', 'spoke', 'talked', 'asked', 'replied', 'answered', 'added', 'continued', 'went on', 'cried', 'sighed', 'thought']

annotated_lines = []

while line != '':
    utterances = re.findall(regex_utterance, line)
    if utterances != []:
        speaker_name = None
        speaker_gender = None
        narrations = re.findall(regex_narration, line)
        if narrations != []:
            # replace narrations with '[X]'
            #line = re.sub(regex_narration_inbetween, ' [X] ', line)
            #line = re.sub(regex_narration_end, '\'\' [X] ', line)
            #tokens = nltk.word_tokenize(narrations[0][0])
            #tagged = nltk.pos_tag(tokens)
            dependencies = [list(parse.triples()) for parse in dep_parser.raw_parse(narrations[0])]
            #dependencies = sum([[list(parse.triples()) for parse in dep_graphs] for dep_graphs in dep_parser.tagged_parse_sents([tagged])],[])
            for ((word1,tag1),dep,(word2,tag2)) in dependencies[0]:
                if tag1.startswith('VB') and dep == 'nsubj':
                    if tag2.startswith('NNP'):
                        speaker_name = word2
                    if word2 in ['he', 'husband', 'man']:
                        speaker_gender = 'M'
                    if word2 in ['she', 'wife', 'lady']:
                        speaker_gender = 'F'
                    print((word1,tag1),dep,(word2,tag2))
            #speaker = " ".join(word for (word, tag) in tagged if tag.startswith('NNP'))
            #entities = nltk.chunk.ne_chunk(tagged)
            #print(entities)
            print("[{}]".format(narrations[0]))
            print(dependencies)
    
        annotated_line = (speaker_name, speaker_gender, line)
        print(annotated_line)
        print('----------------------')
        annotated_lines.append(annotated_line)
    line = raw_text_file.readline()

('said', 'VBD') nsubj ('lady', 'NN')
[ said his lady to him one day, ]
[[(('said', 'VBD'), 'nsubj', ('lady', 'NN')), (('lady', 'NN'), 'nmod:poss', ('his', 'PRP$')), (('lady', 'NN'), 'nmod', ('him', 'PRP')), (('him', 'PRP'), 'case', ('to', 'TO')), (('him', 'PRP'), 'dep', ('day', 'NN')), (('day', 'NN'), 'nummod', ('one', 'CD'))]]
(None, 'F', "``My dear Mr. Bennet,'' said his lady to him one day, ``have you heard that Netherfield Park is let at last?''\n")
----------------------
[ returned she; ]
[[(('returned', 'VBG'), 'dobj', ('she', 'PRP'))]]
(None, None, "``But it is,'' returned she; ``for Mrs. Long has just been here, and she told me all about it.''\n")
----------------------
[ cried his wife impatiently.]
[[(('cried', 'VB'), 'dobj', ('impatiently', 'NN')), (('impatiently', 'NN'), 'nmod:poss', ('his', 'PRP$')), (('impatiently', 'NN'), 'compound', ('wife', 'NN'))]]
(None, None, "``Do not you want to know who has taken it?'' cried his wife impatiently.\n")
----------------------
(None,

[ cried he.  ]
[[(('cried', 'VB'), 'dobj', ('he', 'PRP'))]]
(None, None, "``What can be the meaning of that emphatic exclamation?'' cried he.  ``Do you consider the forms of introduction, and the stress that is laid on them, as nonsense?  I cannot quite agree with you _there_.  What say you, Mary?  for you are a young lady of deep reflection I know, and read great books, and make extracts.''\n")
----------------------
('continued', 'VBD') nsubj ('he', 'PRP')
[ he continued, ]
[[(('continued', 'VBD'), 'nsubj', ('he', 'PRP'))]]
(None, 'M', "``While Mary is adjusting her ideas,'' he continued, ``let us return to Mr. Bingley.''\n")
----------------------
[ cried his wife.]
[[(('cried', 'VB'), 'dobj', ('wife', 'NN')), (('wife', 'NN'), 'nmod:poss', ('his', 'PRP$'))]]
(None, None, "``I am sick of Mr. Bingley,'' cried his wife.\n")
----------------------
(None, None, "``I am sorry to hear _that_; but why did not you tell me so before?  If I had known as much this morning, I certainly would not

('impatiently', 'VB') nsubj ('husband', 'NN')
[ cried her husband impatiently, ]
[[(('cried', 'VB'), 'ccomp', ('impatiently', 'VB')), (('impatiently', 'VB'), 'nsubj', ('husband', 'NN')), (('husband', 'NN'), 'nmod:poss', ('her', 'PRP$'))]]
(None, 'M', "``If he had had any compassion for _me_,'' cried her husband impatiently, ``he would not have danced half so much!  For God's sake, say no more of his partners.  Oh!  that he had sprained his ancle in the first dance!''\n")
----------------------
[ continued Mrs. Bennet, ]
[[(('continued', 'JJ'), 'dep', ('Bennet', 'NNP')), (('Bennet', 'NNP'), 'compound', ('Mrs.', 'NNP'))]]
(None, None, "``Oh!  my dear,'' continued Mrs. Bennet, ``I am quite delighted with him.  He is so excessively handsome!  and his sisters are charming women.  I never in my life saw any thing more elegant than their dresses.  I dare say the lace upon Mrs. Hurst's gown --''\n")
----------------------
('added', 'VBD') nsubj ('she', 'PRP')
[ she added, ]
[[(('added', 'VBD')

[ replied Elizabeth, ]
[[(('Elizabeth', 'NNP'), 'case', ('replied', 'IN'))]]
(None, None, "``That is very true,'' replied Elizabeth, ``and I could easily forgive _his_ pride, if he had not mortified _mine_.''\n")
----------------------
('observed', 'VBD') nsubj ('Mary', 'NNP')
('piqued', 'VBD') nsubj ('who', 'WP')
[ observed Mary, who piqued herself upon the solidity of her reflections, ]
[[(('observed', 'VBD'), 'nsubj', ('Mary', 'NNP')), (('Mary', 'NNP'), 'acl:relcl', ('piqued', 'VBD')), (('piqued', 'VBD'), 'nsubj', ('who', 'WP')), (('piqued', 'VBD'), 'dobj', ('herself', 'PRP')), (('piqued', 'VBD'), 'nmod', ('solidity', 'NN')), (('solidity', 'NN'), 'case', ('upon', 'IN')), (('solidity', 'NN'), 'det', ('the', 'DT')), (('solidity', 'NN'), 'nmod', ('reflections', 'NNS')), (('reflections', 'NNS'), 'case', ('of', 'IN')), (('reflections', 'NNS'), 'nmod:poss', ('her', 'PRP$'))]]
('Mary', None, "``Pride,'' observed Mary, who piqued herself upon the solidity of her reflections, ``is a very com

[ said Miss Lucas. ]
[[(('said', 'VBD'), 'dobj', ('Lucas', 'NNP')), (('Lucas', 'NNP'), 'compound', ('Miss', 'NNP'))]]
(None, None, "``It will be _her_ turn soon to be teazed,'' said Miss Lucas. ``I am going to open the instrument, Eliza, and you know what follows.''\n")
----------------------
('added', 'VBD') nsubj ('she', 'PRP')
[  On Miss Lucas's persevering, however, she added, ]
[[(('added', 'VBD'), 'nmod', ('persevering', 'NN')), (('persevering', 'NN'), 'case', ('On', 'IN')), (('persevering', 'NN'), 'nmod:poss', ('Lucas', 'NNP')), (('Lucas', 'NNP'), 'compound', ('Miss', 'NNP')), (('Lucas', 'NNP'), 'case', ("'s", 'POS')), (('persevering', 'NN'), 'advmod', ('however', 'RB')), (('added', 'VBD'), 'nsubj', ('she', 'PRP'))]]
(None, 'F', '``You are a very strange creature by way of a friend! -- always wanting me to play and sing before any body and every body! -- If my vanity had taken a musical turn, you would have been invaluable, but as it is, I would really rather not sit down before

...

('said', 'VBD') nsubj (',', 'VB')
[ said he, ]
[[(('said', 'VBD'), 'dobj', ('he', 'PRP')), (('said', 'VBD'), 'nsubj', (',', 'VB'))]]
(None, None, "``I have no reason, I assure you,'' said he, ``to be dissatisfied with my reception.  Mr. Darcy seemed much pleased with the attention.  He answered me with the utmost civility, and even paid me the compliment of saying that he was so well convinced of Lady Catherine's discernment as to be certain she could never bestow a favour unworthily.  It was really a very handsome thought.  Upon the whole, I am much pleased with him.''\n")
----------------------
(None, None, "``What is Mr. Darcy to me, pray, that I should be afraid of him?  I am sure we owe him no such particular civility as to be obliged to say nothing _he_ may not like to hear.''\n")
----------------------
(None, None, "``For heaven's sake, madam, speak lower. -- What advantage can it be to you to offend Mr. Darcy? -- You will never recommend yourself to his friend by so doing.''\n"

('cried', 'VBD') nsubj ('she', 'PRP')
[ she cried.  ]
[[(('cried', 'VBD'), 'nsubj', ('she', 'PRP'))]]
(None, 'F', "``You are too hasty, Sir,'' she cried.  ``You forget that I have made no answer.  Let me do it without farther loss of time.  Accept my thanks for the compliment you are paying me, I am very sensible of the honour of your proposals, but it is impossible for me to do otherwise than decline them.''\n")
----------------------
('replied', 'VBD') nsubj ('Collins', 'NNP')
[ replied Mr. Collins, with a formal wave of the hand, ]
[[(('replied', 'VBD'), 'nsubj', ('Collins', 'NNP')), (('Collins', 'NNP'), 'compound', ('Mr.', 'NNP')), (('Collins', 'NNP'), 'nmod', ('wave', 'NN')), (('wave', 'NN'), 'case', ('with', 'IN')), (('wave', 'NN'), 'det', ('a', 'DT')), (('wave', 'NN'), 'amod', ('formal', 'JJ')), (('wave', 'NN'), 'nmod', ('hand', 'NN')), (('hand', 'NN'), 'case', ('of', 'IN')), (('hand', 'NN'), 'det', ('the', 'DT'))]]
('Collins', None, "``I am not now to learn,'' replied Mr. Colli

('finished', 'VBN') nsubj ('she', 'PRP')
[ said he, when she had finished her speech.  ]
[[(('said', 'VBD'), 'dobj', ('he', 'PRP')), (('said', 'VBD'), 'advcl', ('finished', 'VBN')), (('finished', 'VBN'), 'advmod', ('when', 'WRB')), (('finished', 'VBN'), 'nsubj', ('she', 'PRP')), (('finished', 'VBN'), 'aux', ('had', 'VBD')), (('finished', 'VBN'), 'dobj', ('speech', 'NN')), (('speech', 'NN'), 'nmod:poss', ('her', 'PRP$'))]]
(None, 'F', "``I have not the pleasure of understanding you,'' said he, when she had finished her speech.  ``Of what are you talking?''\n")
----------------------
(None, None, "``Of Mr. Collins and Lizzy.  Lizzy declares she will not have Mr. Collins, and Mr. Collins begins to say that he will not have Lizzy.''\n")
----------------------
(None, None, "``And what am I to do on the occasion? -- It seems an hopeless business.''\n")
----------------------
(None, None, "``Speak to Lizzy about it yourself.  Tell her that you insist upon her marrying him.''\n")
-------------

('listened', 'VBD') nsubj ('Elizabeth', 'NNP')
('surprised', 'VBD') nsubj ('suddenness', 'NNS')
('saw', 'VBD') nsubj ('she', 'PRP')
('was', 'VBD') nsubj ('it', 'PRP')
('prevent', 'VB') nsubj ('absence', 'NN')
('being', 'VBG') nsubj ('Bingley', 'NNP')
('cease', 'VB') nsubj ('Jane', 'NNP')
[ To these high flown expressions, Elizabeth listened with all the insensibility of distrust; and though the suddenness of their removal surprised her, she saw nothing in it really to lament; it was not to be supposed that their absence from Netherfield would prevent Mr. Bingley's being there; and as to the loss of their society, she was persuaded that Jane must soon cease to regard it, in the enjoyment of his.]
[[(('listened', 'VBD'), 'nmod', ('expressions', 'NNS')), (('expressions', 'NNS'), 'case', ('To', 'TO')), (('expressions', 'NNS'), 'det', ('these', 'DT')), (('expressions', 'NNS'), 'amod', ('high', 'JJ')), (('expressions', 'NNS'), 'nummod', ('flown', 'CD')), (('listened', 'VBD'), 'nsubj', ('Eliz

StopIteration: 

In [41]:
raw_text_file = open('corpus/PRIDPREJ_NONEWLINE_Organize_v2.txt', 'r+')
[x for x in [raw_text_file.readline() for l in range(25)][3:] if x != '\n']

['However little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered as the rightful property of some one or other of their daughters.\n',
 "``My dear Mr. Bennet,'' said his lady to him one day, ``have you heard that Netherfield Park is let at last?''\n",
 'Mr. Bennet replied that he had not.\n',
 "``But it is,'' returned she; ``for Mrs. Long has just been here, and she told me all about it.''\n",
 'Mr. Bennet made no answer.\n',
 "``Do not you want to know who has taken it?'' cried his wife impatiently.\n",
 "``_You_ want to tell me, and I have no objection to hearing it.''\n",
 'This was invitation enough.\n',
 "``Why, my dear, you must know, Mrs. Long says that Netherfield is taken by a young man of large fortune from the north of England; that he came down on Monday in a chaise and four to see the place, and was so much delighted with it that he agreed wit

In [25]:
sentences = [
    ('"I am hungry"', 'Paul'),
    ('said Paul', 'Narator'),
    ('"I want to buy something to eat"', 'Paul'),
    ('"You should wait until noon"', 'Mirna'),
    ('said Mirna', 'Narator'),
]

LSTM pour chaque phrase -> embedding pour la phrase + 

In [23]:
jar = './stanford-parser-full-2017-06-09/stanford-parser-3.8.0.jar'
model = './stanford-parser-full-2017-06-09/stanford-parser-3.8.0.jar'

dep_parser = stanford.StanfordDependencyParser(model, jar, model_path='edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz', encoding='utf8')

In [34]:
[next((t for t in x.triples() if t[1] == 'nsubj'), None)
 for sentence in sentences
 for x in dep_parser.parse(sentence[0].split())]

[(('hungry"', 'JJ'), 'nsubj', ('"I', 'NNP')),
 (('said', 'VBD'), 'nsubj', ('Paul', 'NNP')),
 (('want', 'VBP'), 'nsubj', ('"I', 'NNP')),
 (('wait', 'VB'), 'nsubj', ('"You', 'NNP')),
 (('said', 'VBD'), 'nsubj', ('Mirna', 'NNP'))]

In [89]:
len(annotated_lines)

503

Indépendant du texte:
- les poids si il y en a ne doivent pas apprendre des mots en eux même mais de la structure, des tags grammaticaux associés aux mots
- on utilise principalement les mots ayant un sens interactionnel (said, replied, look, ...)    
 Un modèle deep learning pourrait apprendre à ne pas tenir compte des autres mots mais cela semble un problème trop compliqué à optimiser 
- on peut se servir des mots pour inférer le sens d'une phrase, un embedding et le rapprocher d'une autre phrase

Ex:
- j'ai faim => [embedding tendance nourriture]
- je vais acheter à manger => [embedding tendance nourriture]

=> produit scalaire donne similarité, et peut être injecté dans le modèle

Options ?
- HMM, avec comme état le numéro de l'énonciateur (Paul=1, Mirna=2, ...), et comme observations des embedding d'énonciation en cours
- 

In [83]:
def make_partition(text):
    part_of_sentences = []
    quote_partition = [s for s in re.sub('\n|\s+', ' ', text+" ").split('"')]
    in_speech = False
    for s in quote_partition:
        part_of_sentences.append((s.strip(), in_speech))
        in_speech = not in_speech
    return [x for x in part_of_sentences if x[0].strip() != '']
make_partition('''
"I am hungry" said Paul
"I want to buy something to eat"
"You should wait until noon" said Mirna
''')

[('I am hungry', True),
 ('said Paul', False),
 ('I want to buy something to eat', True),
 ('You should wait until noon', True),
 ('said Mirna', False)]

Characters involved:
- Paul
- Mirna

Sentences
"I am hungry" [Paul: 0.4, Mirna: 0.6]
said Paul -> 
"I want to buy something to eat"
"You should wait until noon"
said Mirna

In [84]:
def parse(sentence):
    return [next((t for t in x.triples() if t[1] == 'nsubj'), None)
     for x in dep_parser.parse(sentence.split())]

In [86]:
parse('to Mirna said Paul')

[(('said', 'VBD'), 'nsubj', ('Paul', 'NNP'))]